In [1]:
# Instalar PySpark en el Entorno de Colab
!pip install -q pyspark

# Caso 1 · Análisis Hospitalario con PySpark en Entorno Google Collab
**Módulo 7 – Big Data**

**Objetivo:** procesar y analizar el dataset hospitalario completo usando **Spark SQL** y **MLlib**, incluyendo limpieza, consultas SQL y un modelo supervisado de clasificación.

In [2]:
## 1) Inicialización de Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Caso1-Hospital-PySpark").getOrCreate()
print("Spark version:", spark.version)

Spark version: 3.5.1


In [3]:
## 2) Carga del CSV
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Ajustar la ruta del archivo si corresponde
PATH_DATA = "hospital_santiago_280.csv"
df = spark.read.csv(PATH_DATA, header=True, inferSchema=True)
print("Registros cargados:", df.count())
df.printSchema()
df.show(10, truncate=False)

Registros cargados: 280
root
 |-- paciente_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- apellido: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- servicio_clinico: string (nullable = true)
 |-- fecha_ingreso: date (nullable = true)
 |-- diagnostico_principal: string (nullable = true)
 |-- dias_hospitalizacion: integer (nullable = true)
 |-- costo_total: integer (nullable = true)
 |-- medico_tratante: string (nullable = true)
 |-- estado_alta: string (nullable = true)

+-----------+------+---------+----+----+-----------+-----------------+-------------+-------------------------+--------------------+-----------+---------------+-----------+
|paciente_id|nombre|apellido |edad|sexo|comuna     |servicio_clinico |fecha_ingreso|diagnostico_principal    |dias_hospitalizacion|costo_total|medico_tratante|estado_alta|
+-----------+------+---------+----+----+-----------+----------

In [4]:
## 3) Exploración y calidad de datos
# Conteo de nulos por columna
nulls = df.select([count(when(col(c).isNull() | (col(c) == ''), c)).alias(c) for c in df.columns])
nulls.show(truncate=False)

# Duplicados
print("Duplicados:", df.count() - df.dropDuplicates().count())

# Estadísticos de numéricas
num_cols = [c for (c,t) in df.dtypes if t in ["int","bigint","double","float"]]
df.select(num_cols).describe().show()


+-----------+------+--------+----+----+------+----------------+-------------+---------------------+--------------------+-----------+---------------+-----------+
|paciente_id|nombre|apellido|edad|sexo|comuna|servicio_clinico|fecha_ingreso|diagnostico_principal|dias_hospitalizacion|costo_total|medico_tratante|estado_alta|
+-----------+------+--------+----+----+------+----------------+-------------+---------------------+--------------------+-----------+---------------+-----------+
|0          |0     |0       |0   |0   |0     |0               |0            |0                    |0                   |0          |0              |0          |
+-----------+------+--------+----+----+------+----------------+-------------+---------------------+--------------------+-----------+---------------+-----------+

Duplicados: 0
+-------+-----------------+-----------------+--------------------+------------------+
|summary|      paciente_id|             edad|dias_hospitalizacion|       costo_total|
+-------

In [5]:
## 4) Limpieza y preparación
# Estandarizar nombres y manejar nulos
df = df.toDF(*[c.strip().lower().replace(" ", "_") for c in df.columns])
df = df.dropDuplicates()

num_cols = [c for (c,t) in df.dtypes if t in ["int","bigint","double","float"]]
cat_cols = [c for (c,t) in df.dtypes if t not in ["int","bigint","double","float"]]

for c in num_cols:
    df = df.na.fill({c: 0})
for c in cat_cols:
    df = df.na.fill({c: "desconocido"})

df.printSchema(); df.show(5)

root
 |-- paciente_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- apellido: string (nullable = false)
 |-- edad: integer (nullable = false)
 |-- sexo: string (nullable = false)
 |-- comuna: string (nullable = false)
 |-- servicio_clinico: string (nullable = false)
 |-- fecha_ingreso: date (nullable = true)
 |-- diagnostico_principal: string (nullable = false)
 |-- dias_hospitalizacion: integer (nullable = false)
 |-- costo_total: integer (nullable = false)
 |-- medico_tratante: string (nullable = false)
 |-- estado_alta: string (nullable = false)

+-----------+--------+----------+----+----+----------------+----------------+-------------+---------------------+--------------------+-----------+---------------+-----------+
|paciente_id|  nombre|  apellido|edad|sexo|          comuna|servicio_clinico|fecha_ingreso|diagnostico_principal|dias_hospitalizacion|costo_total|medico_tratante|estado_alta|
+-----------+--------+----------+----+----+----------------+--------

In [12]:
## 5) Spark SQL – Consultas
df.createOrReplaceTempView("hospital")

spark.sql("""
SELECT diagnostico_principal, COUNT(*) as total
FROM hospital
GROUP BY diagnostico_principal
ORDER BY total DESC
LIMIT 10
""").show()

spark.sql("""
SELECT servicio_clinico, COUNT(*) as total
FROM hospital
GROUP BY servicio_clinico
ORDER BY total DESC
""").show()

spark.sql("""
SELECT servicio_clinico, ROUND(AVG(dias_hospitalizacion), 2) as estancia_promedio
FROM hospital
GROUP BY servicio_clinico
ORDER BY estancia_promedio DESC
""").show()

+---------------------+-----+
|diagnostico_principal|total|
+---------------------+-----+
|  Enfermedad de Crohn|    2|
| Hepatitis autoinmune|    2|
|  Artritis reumatoide|    2|
| Insuficiencia car...|    2|
|    Lupus eritematoso|    2|
| Síndrome ovario p...|    2|
|  Síndrome de Sjögren|    2|
| Síndrome de Goodp...|    2|
| Esclerosis sistémica|    1|
|    Fractura de radio|    1|
+---------------------+-----+

+-----------------+-----+
| servicio_clinico|total|
+-----------------+-----+
| Medicina Interna|   63|
|    Traumatología|   38|
|      Cardiología|   34|
|       Neurología|   29|
|Gastroenterología|   25|
|     Dermatología|   23|
|        Oncología|   23|
|         Urología|   23|
|      Ginecología|   22|
+-----------------+-----+

+-----------------+-----------------+
| servicio_clinico|estancia_promedio|
+-----------------+-----------------+
|        Oncología|            16.13|
|       Neurología|            11.03|
|      Cardiología|            10.09|
|Gastroenter

In [13]:
## 6) MLlib – Clasificación
from pyspark.sql.functions import col, lit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Selección/creación de etiqueta
candidatas = ["readmision","readmisión","reingreso","criticidad","urgente","label","objetivo","target","readmission","critical"]
LABEL_COL = None
for cand in candidatas:
    if cand in df.columns:
        LABEL_COL = cand; break

if LABEL_COL is None:
    from pyspark.sql import functions as F
    cols_needed = [c for c in ["dias_hospitalizacion","edad"] if c in df.columns]
    if len(cols_needed) == 2:
        q = df.approxQuantile(cols_needed, [0.75], 0.05)
        thr = dict(zip(cols_needed, [vals[0] for vals in q]))
        df = df.withColumn("label",
            ( (col("dias_hospitalizacion") >= thr["dias_hospitalizacion"]) & (col("edad") >= thr["edad"]) ).cast("int"))
        LABEL_COL = "label"
    else:
        df = df.withColumn("label", lit(0).cast("int")); LABEL_COL = "label"

print("LABEL_COL:", LABEL_COL)

# Features
num_cols = [c for (c,t) in df.dtypes if t in ["int","bigint","double","float"] and c != LABEL_COL]
cat_cols = [c for (c,t) in df.dtypes if t not in ["int","bigint","double","float", "date"] and c != LABEL_COL] # Exclude date type

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]
encoder = OneHotEncoder(inputCols=[f"{c}_idx" for c in cat_cols],
                        outputCols=[f"{c}_ohe" for c in cat_cols], handleInvalid="keep")
assembler = VectorAssembler(inputCols=num_cols + [f"{c}_ohe" for c in cat_cols], outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

train, test = df.randomSplit([0.8,0.2], seed=42)

lr = LogisticRegression(featuresCol="features", labelCol=LABEL_COL, maxIter=50)
rf = RandomForestClassifier(featuresCol="features", labelCol=LABEL_COL, numTrees=200, maxDepth=10)

pipe_lr = Pipeline(stages=indexers + [encoder, assembler, scaler, lr])
pipe_rf = Pipeline(stages=indexers + [encoder, assembler, scaler, rf])

m_lr = pipe_lr.fit(train); pred_lr = m_lr.transform(test)
m_rf = pipe_rf.fit(train); pred_rf = m_rf.transform(test)

e_auc = BinaryClassificationEvaluator(labelCol=LABEL_COL, metricName="areaUnderROC")
e_acc = MulticlassClassificationEvaluator(labelCol=LABEL_COL, metricName="accuracy")

print("LR  -> AUC:", e_auc.evaluate(pred_lr), " | Accuracy:", e_acc.evaluate(pred_lr))
print("RF  -> AUC:", e_auc.evaluate(pred_rf), " | Accuracy:", e_acc.evaluate(pred_rf))

# Matrices de confusión rápidas
print("Confusión LR:"); pred_lr.groupBy(LABEL_COL,"prediction").count().orderBy(LABEL_COL,"prediction").show()
print("Confusión RF:"); pred_rf.groupBy(LABEL_COL,"prediction").count().orderBy(LABEL_COL,"prediction").show()

LABEL_COL: label
LR  -> AUC: 0.8796296296296298  | Accuracy: 0.8809523809523809
RF  -> AUC: 0.986111111111111  | Accuracy: 0.9047619047619048
Confusión LR:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|   35|
|    0|       1.0|    1|
|    1|       0.0|    4|
|    1|       1.0|    2|
+-----+----------+-----+

Confusión RF:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|   36|
|    1|       0.0|    4|
|    1|       1.0|    2|
+-----+----------+-----+



In [14]:
## 7) Guardado de modelos y métricas
m_lr.write().overwrite().save("models/caso1_lr_pipeline")
m_rf.write().overwrite().save("models/caso1_rf_pipeline")

import json, os
metrics = {
    "lr": {"auc": e_auc.evaluate(pred_lr), "accuracy": e_acc.evaluate(pred_lr)},
    "rf": {"auc": e_auc.evaluate(pred_rf), "accuracy": e_acc.evaluate(pred_rf)}
}
os.makedirs("artifacts", exist_ok=True)
with open("artifacts/caso1_metrics.json","w") as f:
    json.dump(metrics, f, indent=2)

print("Modelos guardados en 'models/' y métricas en 'artifacts/caso1_metrics.json'")



Modelos guardados en 'models/' y métricas en 'artifacts/caso1_metrics.json'
